In [1]:
import openeo
from openeo.processes import run_udf

In [2]:
connection = openeo.connect("https://openeo.cloud")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeocloud.vito.be/openeo/1.0.0/' with OidcBearerAuth>

In [14]:
# Load high-res and low-res data from OpenEO Platform
aoi = {
    "type": "Polygon",
    "coordinates": [[
      [
        8.1733155,
        56.554395
      ],
      [
        8.1733155,
        56.554395
      ],
      [
        8.1671337,
        56.4854867
      ],
      [
        8.2378811,
        56.5051875
      ],
      [
        8.2323862,
        56.5464504
      ],
      [
        8.1733155,
        56.554395
      ]
    ]]
}
start_date = "2024-09-18"
end_date = "2024-09-18"

sentinel2_l2a_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=aoi,
    temporal_extent=[start_date, end_date],
    #bands=["B04", "B03", "B02"], 
)
sentinel3_lst = connection.load_collection(
    "SENTINEL3_SLSTR",
    spatial_extent=aoi,
   temporal_extent=[start_date, end_date],
   bands=['S8'],  
)


In [12]:
udf = openeo.UDF.from_file("../openeo_workflows/dms/udf_pass_filepaths.py")

In [5]:
sentinel2_l2a_cube = sentinel2_l2a_cube.save_result(format="GTiff")
sentinel3_lst = sentinel3_lst.save_result(format="GTiff")

In [6]:
sentinel2_l2a_job = sentinel2_l2a_cube.create_job().start_and_wait()
sentinel3_lst_job = sentinel3_lst.create_job().start_and_wait()

0:00:00 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': send 'start'
0:00:19 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': created (progress 0%)
0:00:25 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:00:31 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:00:40 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:00:51 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:01:03 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:01:18 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:01:38 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:02:03 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:02:33 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': queued (progress 0%)
0:03:11 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': running (progress N/A)
0:03:57 Job 'vito-j-241105eaace64475a62ddfe40c9e1a59': running (progress N/A)
0:04:56 Job 'vi

In [8]:
sentinel2_l2a_file_path = sentinel2_l2a_job.get_results().get_assets()[0].name
sentinel3_lst_file_path = sentinel3_lst_job.get_results().get_assets()[0].name

In [9]:
context = {"high_res_file":sentinel2_l2a_file_path, "low_res_file":sentinel3_lst_file_path}

In [17]:
udf_process = run_udf(data=sentinel2_l2a_cube,udf=udf, context=context, runtime="python")

In [21]:
udf_process.save_result(format="GTiff").create_job().start_and_wait()

AttributeError: 'ProcessBuilder' object has no attribute 'create_job'

In [18]:
connection.create_job(process_graph=udf_process)

Preflight process graph validation failed: Object of type UDF is not JSON serializable


TypeError: Object of type UDF is not JSON serializable

In [67]:
dms_res = dms_res.save_result(format="GTiff")
dms_res_job = dms_res.create_job().start_and_wait()

0:00:00 Job 'vito-j-24110495427c4b0a8144937683a2841b': send 'start'
0:00:17 Job 'vito-j-24110495427c4b0a8144937683a2841b': created (progress 0%)
0:00:22 Job 'vito-j-24110495427c4b0a8144937683a2841b': queued (progress 0%)
0:00:29 Job 'vito-j-24110495427c4b0a8144937683a2841b': queued (progress 0%)
0:00:37 Job 'vito-j-24110495427c4b0a8144937683a2841b': queued (progress 0%)
0:00:47 Job 'vito-j-24110495427c4b0a8144937683a2841b': queued (progress 0%)
0:00:59 Job 'vito-j-24110495427c4b0a8144937683a2841b': queued (progress 0%)
0:01:15 Job 'vito-j-24110495427c4b0a8144937683a2841b': queued (progress 0%)
0:01:34 Job 'vito-j-24110495427c4b0a8144937683a2841b': running (progress N/A)
0:01:58 Job 'vito-j-24110495427c4b0a8144937683a2841b': running (progress N/A)
0:02:28 Job 'vito-j-24110495427c4b0a8144937683a2841b': running (progress N/A)
0:03:06 Job 'vito-j-24110495427c4b0a8144937683a2841b': running (progress N/A)
0:03:52 Job 'vito-j-24110495427c4b0a8144937683a2841b': running (progress N/A)
0:04:51 J

JobFailedException: Batch job 'vito-j-24110495427c4b0a8144937683a2841b' didn't finish successfully. Status: error (after 0:05:51).

In [ ]:
dms_res = sentinel2_l2a_cube.apply(process=udf, context = context)